# Bridge MCP servers to ACP

This cookbook demonstrates how to use any-agent's MCP-to-ACP bridge to expose Model Context Protocol (MCP) servers as Agent Connect Protocol (ACP) compatible services.

## Why Bridge MCP to ACP?

- **Protocol Interoperability**: Make MCP tools available to Linux Foundation's AGNTCY ecosystem
- **Enterprise Integration**: Connect local tools with enterprise agent systems
- **Identity Support**: Leverage AGNTCY Identity (W3C DIDs) for secure tool access
- **Standards Compliance**: Support for W3C Decentralized Identifiers and Verifiable Credentials

## Prerequisites

You'll need to install the ACP SDK:

```bash
pip install agntcy-acp
```

In [ ]:
# For Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

## Example 1: Basic MCP to ACP Bridge

Let's start by bridging a simple MCP time server to ACP:

In [ ]:
import asyncio
from any_agent.config import MCPStdio
from any_agent.serving import (
    MCPToACPBridgeConfig,
    serve_mcp_as_acp_async,
)

async def bridge_time_server():
    # Configure MCP server
    mcp_config = MCPStdio(
        command="uvx",
        args=["mcp-server-time"],
        tools=["get_current_time"],
    )
    
    # Configure bridge
    bridge_config = MCPToACPBridgeConfig(
        mcp_config=mcp_config,
        port=8090,
        endpoint="/time-bridge",
        server_name="time-server",
        organization="any-agent-demo",
        version="1.0.0",
    )
    
    # Start the bridge
    print("Starting MCP to ACP bridge...")
    bridge_handle = await serve_mcp_as_acp_async(mcp_config, bridge_config)
    
    print(f"✓ MCP server bridged to ACP at: http://localhost:8090/time-bridge")
    print(f"  - ACP agents endpoint: http://localhost:8090/time-bridge/agents")
    print(f"  - Stateless runs endpoint: http://localhost:8090/time-bridge/runs/stateless")
    
    return bridge_handle

# Start the bridge (run in background)
bridge_task = asyncio.create_task(bridge_time_server())
bridge_handle = await bridge_task
print("\nBridge is running!")

## Example 2: Using the Bridged MCP Tool via ACP

Now let's use the bridged MCP tool through the ACP protocol:

In [ ]:
from agntcy_acp import ACPAsyncClient, RunCreateStateless

async def use_bridged_tool():
    # Create ACP client
    client = ACPAsyncClient(base_url="http://localhost:8090/time-bridge")
    
    # List available agents
    print("Listing available agents...")
    agents = await client.list_agents()
    for agent in agents:
        print(f"  - Agent ID: {agent.id}")
        print(f"    Name: {agent.name}")
        print(f"    Description: {agent.description}")
    
    # Create a stateless run to get the current time
    print("\nCalling get_current_time tool...")
    run_request = RunCreateStateless(
        config={
            "tool": "get_current_time",
            "args": {}
        }
    )
    
    result = await client.create_stateless_run(
        agent_id="mcp-bridge-time-server",
        run_request=run_request
    )
    
    print(f"\nResult:")
    print(f"  Status: {result.status}")
    print(f"  Output: {result.output}")

await use_bridged_tool()

## Example 3: Bridge with AGNTCY Identity

This example shows how to configure the bridge with AGNTCY Identity support:

In [ ]:
async def bridge_with_identity():
    # Configure MCP server
    mcp_config = MCPStdio(
        command="uvx",
        args=["mcp-server-filesystem"],
        tools=["read_file", "write_file", "list_files"],
    )
    
    # Configure bridge with identity
    bridge_config = MCPToACPBridgeConfig(
        mcp_config=mcp_config,
        port=8091,
        endpoint="/fs-bridge",
        server_name="filesystem-server",
        # W3C DID for AGNTCY Identity
        identity_id="did:agntcy:example:filesystem-bridge-123",
        organization="secure-tools-org",
        version="1.0.0",
    )
    
    # Start the bridge
    print("Starting MCP to ACP bridge with AGNTCY Identity...")
    bridge_handle = await serve_mcp_as_acp_async(mcp_config, bridge_config)
    
    print(f"\n✓ Secure MCP server bridged to ACP")
    print(f"  - Endpoint: http://localhost:8091/fs-bridge")
    print(f"  - Identity DID: {bridge_config.identity_id}")
    print(f"  - Organization: {bridge_config.organization}")
    
    return bridge_handle

# Uncomment to run
# secure_bridge = await bridge_with_identity()

## Example 4: Multiple MCP Tools via ACP

When an MCP server has multiple tools, you can specify which tool to call:

In [ ]:
async def call_specific_tool():
    client = ACPAsyncClient(base_url="http://localhost:8091/fs-bridge")
    
    # List files in current directory
    print("Listing files...")
    list_request = RunCreateStateless(
        config={
            "tool": "list_files",
            "args": {
                "path": "."
            }
        }
    )
    
    result = await client.create_stateless_run(
        agent_id="mcp-bridge-filesystem-server",
        run_request=list_request
    )
    
    print(f"Files: {result.output}")
    
    # Read a specific file
    print("\nReading README.md...")
    read_request = RunCreateStateless(
        config={
            "tool": "read_file",
            "args": {
                "path": "README.md"
            }
        }
    )
    
    result = await client.create_stateless_run(
        agent_id="mcp-bridge-filesystem-server",
        run_request=read_request
    )
    
    print(f"Content preview: {str(result.output)[:200]}...")

# Uncomment if filesystem bridge is running
# await call_specific_tool()

## Clean Up

Stop the bridge servers when done:

In [ ]:
# Stop the bridge
if 'bridge_handle' in locals() and bridge_handle:
    bridge_handle.server.should_exit = True
    await asyncio.sleep(1)
    print("Bridge stopped.")

## Summary

In this cookbook, we've demonstrated:

1. **Basic Bridge Setup**: How to expose an MCP server via ACP protocol
2. **Using Bridged Tools**: How to call MCP tools through the ACP client
3. **Identity Integration**: Adding AGNTCY Identity (W3C DIDs) to bridges
4. **Multi-Tool Support**: Working with MCP servers that have multiple tools

The MCP-to-ACP bridge enables seamless interoperability between Mozilla's MCP ecosystem and Linux Foundation's AGNTCY platform, supporting enterprise integration scenarios with secure identity management.

## Next Steps

- Learn more about [AGNTCY Identity](https://spec.identity.agntcy.org/)
- Explore [W3C DIDs](https://www.w3.org/TR/did-core/) and [Verifiable Credentials](https://www.w3.org/TR/vc-data-model/)
- Check out the [Linux Foundation AGNTCY Project](https://www.linuxfoundation.org/press/linux-foundation-welcomes-the-agntcy-project-to-standardize-open-multi-agent-system-infrastructure-and-break-down-ai-agent-silos)